# SageMaker Object Detection model transform

This is the repeat error message for the [issue 15181](https://github.com/apache/incubator-mxnet/issues/15181) : `No conversion function registered for op type _contrib_MultiBoxPrior yet`

In [1]:
%%bash
git clone https://github.com/apache/incubator-mxnet

Cloning into 'incubator-mxnet'...


In [2]:
import boto3
sm = boto3.client('sagemaker')
response = sm.describe_training_job(TrainingJobName='object-detection-2020-04-22-09-25-36-709')
model_path = response['ModelArtifacts']['S3ModelArtifacts']
model_path

's3://beyoung-sagemaker/coco-object-detection-20200422/output/object-detection-2020-04-22-09-25-36-709/output/model.tar.gz'

In [3]:
!aws s3 cp {model_path} ./model.tar.gz
!tar -xvf ./model.tar.gz -C ./incubator-mxnet/example/ssd/model/
!ls ./incubator-mxnet/example/ssd/model/

download: s3://beyoung-sagemaker/coco-object-detection-20200422/output/object-detection-2020-04-22-09-25-36-709/output/model.tar.gz to ./model.tar.gz
hyperparams.json
model_algo_1-0000.params
model_algo_1-symbol.json
hyperparams.json  model_algo_1-0000.params  model_algo_1-symbol.json  README.md


In [4]:
# change notebook working directory
%cd ./incubator-mxnet/example/ssd
#deploy model as checkpoint format
!python deploy.py --network resnet50 --prefix model/model_algo_1 --num-class 80 --data-shape 512

/home/ec2-user/SageMaker/myAWSStudyBlog/sagemaker-image-classification-detection/8-tensort/incubator-mxnet/example/ssd
[02:41:01] src/nnvm/legacy_json_util.cc:209: Loading symbol saved by previous version v1.4.1. Attempting to upgrade...
[02:41:01] src/nnvm/legacy_json_util.cc:217: Symbol successfully upgraded!
Saved model: model/deploy_model_algo_1-0000.params
Saved symbol: model/deploy_model_algo_1-symbol.json


In [5]:
## ref: https://github.com/apache/incubator-mxnet/issues/15181#issuecomment-519307238

import numpy as np
from mxnet.contrib import onnx as onnx_mxnet
import logging
logging.basicConfig(level=logging.INFO)

# Downloaded input symbol and params files
sym = './model/deploy_model_algo_1-symbol.json'
params = './model/deploy_model_algo_1-0000.params'

# Standard Imagenet input - 3 channels, 512*512
input_shape = (1,3,512,512)

# Path of the output file
onnx_file = './mxnet_exported.onnx'

converted_model_path = onnx_mxnet.export_model(sym, params, [input_shape], np.float32, onnx_file)

INFO:root:Converting json and weight file to sym and params


AttributeError: No conversion function registered for op type _contrib_MultiBoxPrior yet.